In [1]:
import os
import numpy as np
import pandas as pd
import hickle as hkl

from collections import defaultdict

from sklearn.metrics import roc_auc_score, r2_score, accuracy_score, explained_variance_score, mean_absolute_error, mean_squared_error

import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Masking, Dropout, LSTM, Dense, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

from IPython.core.display import display, HTML



# Hide GPU from visible devices

#'''
tf.config.set_visible_devices([], 'GPU')
print(f'CUDA GPU AVAILABLE: {tf.test.is_gpu_available(cuda_only=True)}')
'''
THREADS = 8
os.environ['OMP_NUM_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTEROP_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTRAOP_THREADS'] = str(THREADS)
tf.config.threading.set_inter_op_parallelism_threads(THREADS)
tf.config.threading.set_intra_op_parallelism_threads(THREADS)
tf.config.set_soft_device_placement(True)
'''

#'''
display(HTML("<style>.container { width:100% !important; }</style>"))
#'''

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
CUDA GPU AVAILABLE: True


In [2]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean((y_pred - y_true) / y_true)

In [3]:
N_SPLITS = 5
N_STEPS = 20
N_DAYS = 60
DATA_DIR = f'data/processed_data/cv_data__{N_SPLITS}_class_id_folds__{N_STEPS}_steps__{N_DAYS}_days'

In [4]:
results = []

for i in range(N_SPLITS):

    # Load one partition of folds
    data = hkl.load(f'{DATA_DIR}/fold_{i}.hkl')
    #action_training_input = data['action_training_input']
    #recurrent_training_input = data['recurrent_training_input']
    prior_training_input = data['prior_training_input']
    completion_training_target = data['completion_training_target']
    problems_training_target = data['problems_training_target']
    training_target_sequence = data['training_target_sequence']
    #action_testing_input = data['action_testing_input']
    #recurrent_testing_input = data['recurrent_testing_input']
    prior_testing_input = data['prior_testing_input']
    completion_testing_target = data['completion_testing_target']
    problems_testing_target = data['problems_testing_target']
    testing_target_sequence = data['testing_target_sequence']
    
    # Clear session so models don't pile up
    keras.backend.clear_session()

    # Create model
    
    # Action Model
    #action_input_layer = Input(shape=action_training_input[0].shape, name='action')
    #action_model = Dropout(rate=0.5)(action_input_layer)
    #action_model = LSTM(units=64, return_sequences=False, activation='relu', dropout=0.5, recurrent_dropout=0.5)(action_model)
    
    # Recurrent Input
    #recurrent_input_layer = Input(shape=recurrent_training_input[0].shape, name='recurrent')
    #recurrent_model = Masking(mask_value=0.0)(recurrent_input_layer)
    #recurrent_model = Dropout(rate=0.5)(recurrent_model)
    #recurrent_model = LSTM(units=64, return_sequences=False, dropout=0.5, recurrent_dropout=0.5)(recurrent_model)
    
    # Prior Input
    prior_input_layer = Input(shape=prior_training_input[0].shape, name='prior')
    prior_model = Dropout(rate=0.5)(prior_input_layer)
    prior_model = Dense(units=64, activation='sigmoid')(prior_model)
    
    # Combined Model
    #model = Concatenate()([action_model, recurrent_model, prior_model])
    model = Dropout(rate=0.5)(prior_model)
    
    # Combined Output
    completion_output_layer = Dense(units=1, activation='sigmoid', name='completion')(model)
    problems_output_layer = Dense(units=1, activation='linear', name='problems')(model)
    
    combined_model = Model(prior_input_layer, [completion_output_layer, problems_output_layer])
    combined_model.compile(optimizer='adam', loss={'completion': 'binary_crossentropy', 'problems': 'mse'})

    # Train model
    es = [EarlyStopping(monitor='val_loss', patience=10, min_delta=0, restore_best_weights=True)]
    weights = {'completion': np.ones_like(completion_training_target) * 16, 'problems': completion_training_target}
    combined_model.fit(x={'prior': prior_training_input},
                       y={'completion': completion_training_target, 'problems': problems_training_target},
                       epochs=1000,
                       validation_split=0.25,
                       callbacks=es,
                       sample_weight=weights,
                       verbose=1)
    
    # Store model predictions
    completion_testing_output, problems_testing_output = combined_model.predict({'prior': prior_testing_input})
    df = pd.DataFrame(zip(np.ones_like(testing_target_sequence).flatten() * i, 
                          testing_target_sequence.flatten(), 
                          completion_testing_target.flatten(), 
                          problems_testing_target.flatten(), 
                          completion_testing_output.flatten(), 
                          problems_testing_output.flatten()), 
                      columns = ['fold', 
                                 'target_sequence', 
                                 'completion_target', 
                                 'problems_target', 
                                 'completion_prediction', 
                                 'problems_prediction'])
    results.append(df)
    

pd.concat(results).to_csv('priors_cross_validation_results.csv', index=False)

Epoch 1/1000
162/162 [==============================] - 1s 4ms/step - loss: 42.9157 - completion_loss: 9.8565 - problems_loss: 33.0592 - val_loss: 17.2999 - val_completion_loss: 9.5700 - val_problems_loss: 7.7299
Epoch 2/1000
162/162 [==============================] - 0s 3ms/step - loss: 34.7213 - completion_loss: 9.0341 - problems_loss: 25.6872 - val_loss: 16.4330 - val_completion_loss: 9.2794 - val_problems_loss: 7.1537
Epoch 3/1000
162/162 [==============================] - 0s 3ms/step - loss: 34.0887 - completion_loss: 8.9718 - problems_loss: 25.1169 - val_loss: 16.3274 - val_completion_loss: 9.2697 - val_problems_loss: 7.0577
Epoch 4/1000
162/162 [==============================] - 1s 3ms/step - loss: 33.8788 - completion_loss: 8.7775 - problems_loss: 25.1013 - val_loss: 16.2099 - val_completion_loss: 9.2093 - val_problems_loss: 7.0006
Epoch 5/1000
162/162 [==============================] - 0s 3ms/step - loss: 33.4541 - completion_loss: 8.7037 - problems_loss: 24.7504 - val_loss: 1

162/162 [==============================] - 1s 8ms/step - loss: 34.5270 - completion_loss: 8.7836 - problems_loss: 25.7434 - val_loss: 16.5092 - val_completion_loss: 9.1029 - val_problems_loss: 7.4063
Epoch 9/1000
162/162 [==============================] - 1s 8ms/step - loss: 33.9734 - completion_loss: 8.7499 - problems_loss: 25.2235 - val_loss: 16.4455 - val_completion_loss: 9.0154 - val_problems_loss: 7.4301
Epoch 10/1000
162/162 [==============================] - 1s 8ms/step - loss: 34.0430 - completion_loss: 8.6840 - problems_loss: 25.3590 - val_loss: 16.5199 - val_completion_loss: 9.0919 - val_problems_loss: 7.4281
Epoch 11/1000
162/162 [==============================] - 1s 8ms/step - loss: 34.0196 - completion_loss: 8.7020 - problems_loss: 25.3176 - val_loss: 16.6054 - val_completion_loss: 9.1104 - val_problems_loss: 7.4950
Epoch 12/1000
162/162 [==============================] - 1s 8ms/step - loss: 34.0002 - completion_loss: 8.5635 - problems_loss: 25.4367 - val_loss: 16.3736 - v

162/162 [==============================] - 1s 5ms/step - loss: 16.7856 - completion_loss: 8.6861 - problems_loss: 8.0995 - val_loss: 15.9831 - val_completion_loss: 9.4844 - val_problems_loss: 6.4987
Epoch 18/1000
162/162 [==============================] - 1s 5ms/step - loss: 16.2856 - completion_loss: 8.4406 - problems_loss: 7.8450 - val_loss: 15.7922 - val_completion_loss: 9.2869 - val_problems_loss: 6.5054
Epoch 19/1000
162/162 [==============================] - 1s 5ms/step - loss: 16.5495 - completion_loss: 8.6663 - problems_loss: 7.8832 - val_loss: 15.9685 - val_completion_loss: 9.4647 - val_problems_loss: 6.5037
Epoch 20/1000
162/162 [==============================] - 1s 6ms/step - loss: 16.4709 - completion_loss: 8.6133 - problems_loss: 7.8576 - val_loss: 15.9635 - val_completion_loss: 9.4596 - val_problems_loss: 6.5039
Epoch 21/1000
162/162 [==============================] - 1s 6ms/step - loss: 16.3536 - completion_loss: 8.5851 - problems_loss: 7.7685 - val_loss: 15.9391 - val_c

162/162 [==============================] - 1s 5ms/step - loss: 33.2628 - completion_loss: 8.5865 - problems_loss: 24.6763 - val_loss: 14.5271 - val_completion_loss: 8.6692 - val_problems_loss: 5.8578
Epoch 21/1000
162/162 [==============================] - 1s 5ms/step - loss: 33.2309 - completion_loss: 8.4670 - problems_loss: 24.7639 - val_loss: 14.5151 - val_completion_loss: 8.7825 - val_problems_loss: 5.7326
Epoch 22/1000
162/162 [==============================] - 1s 4ms/step - loss: 33.0608 - completion_loss: 8.3790 - problems_loss: 24.6818 - val_loss: 14.4783 - val_completion_loss: 8.7330 - val_problems_loss: 5.7452
Epoch 23/1000
162/162 [==============================] - 1s 5ms/step - loss: 32.9320 - completion_loss: 8.5604 - problems_loss: 24.3716 - val_loss: 14.5808 - val_completion_loss: 8.8083 - val_problems_loss: 5.7725
Epoch 24/1000
162/162 [==============================] - 1s 4ms/step - loss: 33.4685 - completion_loss: 8.4385 - problems_loss: 25.0299 - val_loss: 14.5354 - 

162/162 [==============================] - 1s 5ms/step - loss: 32.7832 - completion_loss: 8.7149 - problems_loss: 24.0683 - val_loss: 18.2427 - val_completion_loss: 8.2920 - val_problems_loss: 9.9507
Epoch 21/1000
162/162 [==============================] - 1s 5ms/step - loss: 33.1425 - completion_loss: 8.5858 - problems_loss: 24.5566 - val_loss: 18.2179 - val_completion_loss: 8.2378 - val_problems_loss: 9.9802
Epoch 22/1000
162/162 [==============================] - 1s 6ms/step - loss: 32.9753 - completion_loss: 8.5806 - problems_loss: 24.3947 - val_loss: 18.3558 - val_completion_loss: 8.2538 - val_problems_loss: 10.1021
Epoch 23/1000
162/162 [==============================] - 1s 5ms/step - loss: 33.0210 - completion_loss: 8.6195 - problems_loss: 24.4015 - val_loss: 18.1677 - val_completion_loss: 8.2576 - val_problems_loss: 9.9101
Epoch 24/1000
162/162 [==============================] - 1s 6ms/step - loss: 32.7759 - completion_loss: 8.6083 - problems_loss: 24.1676 - val_loss: 18.1440 -

162/162 [==============================] - 1s 6ms/step - loss: 30.3149 - completion_loss: 8.5777 - problems_loss: 21.7372 - val_loss: 17.5150 - val_completion_loss: 8.2680 - val_problems_loss: 9.2470
Epoch 59/1000
162/162 [==============================] - 1s 5ms/step - loss: 31.8340 - completion_loss: 8.4918 - problems_loss: 23.3422 - val_loss: 17.5545 - val_completion_loss: 8.3011 - val_problems_loss: 9.2534
Epoch 60/1000
162/162 [==============================] - 1s 6ms/step - loss: 30.4608 - completion_loss: 8.6002 - problems_loss: 21.8606 - val_loss: 17.5216 - val_completion_loss: 8.2424 - val_problems_loss: 9.2792
Epoch 61/1000
162/162 [==============================] - 1s 6ms/step - loss: 31.7830 - completion_loss: 8.5158 - problems_loss: 23.2672 - val_loss: 17.5349 - val_completion_loss: 8.2892 - val_problems_loss: 9.2456
Epoch 62/1000
162/162 [==============================] - 1s 7ms/step - loss: 30.7974 - completion_loss: 8.5389 - problems_loss: 22.2585 - val_loss: 17.4591 - 

162/162 [==============================] - 1s 7ms/step - loss: 28.9544 - completion_loss: 8.6454 - problems_loss: 20.3090 - val_loss: 16.7914 - val_completion_loss: 8.3740 - val_problems_loss: 8.4174
Epoch 97/1000
162/162 [==============================] - 1s 7ms/step - loss: 28.7657 - completion_loss: 8.5900 - problems_loss: 20.1758 - val_loss: 16.7999 - val_completion_loss: 8.3974 - val_problems_loss: 8.4025
Epoch 98/1000
162/162 [==============================] - 1s 6ms/step - loss: 26.0157 - completion_loss: 8.5622 - problems_loss: 17.4535 - val_loss: 16.7713 - val_completion_loss: 8.4223 - val_problems_loss: 8.3490
Epoch 99/1000
162/162 [==============================] - 1s 8ms/step - loss: 25.4961 - completion_loss: 8.5213 - problems_loss: 16.9747 - val_loss: 16.7473 - val_completion_loss: 8.4378 - val_problems_loss: 8.3096
Epoch 100/1000
162/162 [==============================] - 1s 6ms/step - loss: 27.2328 - completion_loss: 8.5509 - problems_loss: 18.6819 - val_loss: 16.7193 -

162/162 [==============================] - 1s 3ms/step - loss: 26.5041 - completion_loss: 8.5351 - problems_loss: 17.9690 - val_loss: 16.0768 - val_completion_loss: 8.4694 - val_problems_loss: 7.6075
Epoch 135/1000
162/162 [==============================] - 1s 3ms/step - loss: 26.2484 - completion_loss: 8.6492 - problems_loss: 17.5992 - val_loss: 16.0531 - val_completion_loss: 8.4569 - val_problems_loss: 7.5961
Epoch 136/1000
162/162 [==============================] - 1s 4ms/step - loss: 25.8411 - completion_loss: 8.6007 - problems_loss: 17.2404 - val_loss: 16.0578 - val_completion_loss: 8.4756 - val_problems_loss: 7.5822
Epoch 137/1000
162/162 [==============================] - 1s 4ms/step - loss: 26.4028 - completion_loss: 8.5329 - problems_loss: 17.8699 - val_loss: 16.0166 - val_completion_loss: 8.4496 - val_problems_loss: 7.5670
Epoch 138/1000
162/162 [==============================] - 1s 3ms/step - loss: 27.3234 - completion_loss: 8.5134 - problems_loss: 18.8100 - val_loss: 16.015

162/162 [==============================] - 1s 7ms/step - loss: 27.5162 - completion_loss: 8.5142 - problems_loss: 19.0020 - val_loss: 15.7892 - val_completion_loss: 8.4820 - val_problems_loss: 7.3072
Epoch 173/1000
162/162 [==============================] - 1s 6ms/step - loss: 27.0721 - completion_loss: 8.5506 - problems_loss: 18.5215 - val_loss: 15.7952 - val_completion_loss: 8.4776 - val_problems_loss: 7.3176
Epoch 174/1000
162/162 [==============================] - 1s 7ms/step - loss: 27.0624 - completion_loss: 8.5642 - problems_loss: 18.4981 - val_loss: 15.8206 - val_completion_loss: 8.4937 - val_problems_loss: 7.3269
Epoch 175/1000
162/162 [==============================] - 1s 7ms/step - loss: 27.3946 - completion_loss: 8.5173 - problems_loss: 18.8773 - val_loss: 15.7607 - val_completion_loss: 8.4657 - val_problems_loss: 7.2950
Epoch 176/1000
162/162 [==============================] - 1s 7ms/step - loss: 22.6638 - completion_loss: 8.6198 - problems_loss: 14.0440 - val_loss: 15.821

In [5]:
# Load the data

exp_norm_map = pd.read_csv('data/experiment_information/exp_norm_map.csv')
results = pd.read_csv('priors_cross_validation_results.csv')

In [6]:
# Evaluate the results

norm_exp_dict = defaultdict(lambda: 'None')
for n, e in zip(exp_norm_map['normal_id'], exp_norm_map['experiment_id']):
    norm_exp_dict[n] = e
results['experiment_sequence'] = results['target_sequence'].map(norm_exp_dict)

metrics = []

# Calculate the metrics for each sequence
for sequence, df in results.groupby('target_sequence'):
    
    completion_target = df['completion_target']
    
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([sequence, 
                    norm_exp_dict[sequence], 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for each fold
for fold, df in results.groupby('fold'):
    
    completion_target = df['completion_target']
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([f'fold_{fold}', 
                    'None', 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for everything combined
completion_target = results['completion_target']
completion_prediction = results['completion_prediction']
completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
completion_mse = mean_squared_error(completion_target, completion_prediction)

problems_target = results[results['completion_target'] == 1]['problems_target']
problems_prediction = results[results['completion_target'] == 1]['problems_prediction']
problems_mae = mean_absolute_error(problems_target, problems_prediction)
problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
problems_mse = mean_squared_error(problems_target, problems_prediction)

metrics.append(['all_target_data', 
                'None', 
                len(results), 
                completion_auc, 
                completion_acc, 
                completion_r2, 
                completion_ev, 
                completion_mse, 
                problems_mae, 
                problems_mape, 
                problems_r2, 
                problems_ev, 
                problems_mse])

metrics = pd.DataFrame(metrics, 
                       columns=['sequence_id', 
                                'experiment_id', 
                                'sample_size', 
                                'completion_auc', 
                                'completion_acc', 
                                'completion_r2', 
                                'completion_ev', 
                                'completion_mse', 
                                'problems_mae', 
                                'problems_mape', 
                                'problems_r2', 
                                'problems_ev', 
                                'problems_mse'])

metrics.to_csv('priors_cross_validation_metrics.csv', index=False)

In [7]:
metrics.iloc[-6:]

,sequence_id,experiment_id,sample_size,completion_auc,completion_acc,completion_r2,completion_ev,completion_mse,problems_mae,problems_mape,problems_r2,problems_ev,problems_mse
26,fold_0,None,1722,0.753775,0.713705,0.150259,0.155281,0.183820,2.422173,0.179625,0.054021,0.067211,14.808143
27,fold_1,None,1722,0.772880,0.749129,0.152327,0.152461,0.167090,2.128310,0.323088,-0.033945,-0.002039,7.695243
28,fold_2,None,1721,0.783133,0.762929,0.160668,0.160669,0.157557,3.083150,0.226603,0.003402,0.014461,80.908688
29,fold_3,None,1721,0.744221,0.743754,0.138158,0.138582,0.171316,2.184816,0.245428,0.081247,0.081640,11.008517
30,fold_4,None,1721,0.755020,0.727484,0.135306,0.135648,0.174790,1.887417,0.169626,0.079246,0.081675,7.834173
31,all_target_data,None,8607,0.761828,0.739398,0.149181,0.149788,0.170916,2.345709,0.229668,0.027327,0.028873,24.943782
